In [17]:
import xarray as xr
import fsspec
import s3fs
import os
import matplotlib.pyplot as plt
import dask
import rasterio
from dask.distributed import Client, LocalCluster, progress
import datetime
import tempfile
import boto3

In [2]:
env = dict(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR', 
           AWS_NO_SIGN_REQUEST='YES',
           GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
           GDAL_SWATH_SIZE='200000000',
           VSI_CURL_CACHE_SIZE='200000000')
os.environ.update(env)

In [54]:
def convert_full_date_to_continous_day(year, month, day):
    """
    Helper function if you wish to use month, day vs julian day
    """
    return datetime.datetime(year, month, day).timetuple().tm_yday

def get_geo_data(year, month, day1, day2):
    """
    returns goes17 data in that year, month from day1 to day2 in a chunked xarray
    """
    if day1 >= day2:
        raise Exception("Must provide a date range where day1 < day2")
        
    fs = s3fs.S3FileSystem(anon=True)
    files = []

    for day_iterator in range(day2 - day1):
        filepath = "s3://noaa-goes17/ABI-L2-SSTF/%s/%s/*/*.nc" % (str(year).zfill(4), str(day_iterator).zfill(3)) 
        temp_files, var = fs.glob(filepath), 'SST'
        files += temp_files
    
    if len(files) < 1:
        raise Exception("No files found")

    file_ob = [fs.open(file) for file in files]
    ds = xr.open_mfdataset(file_ob, combine='nested', concat_dim='time', parallel=True).chunk({'time': 10})
    
    return ds


In [46]:
d = get_geo_data(2020, 11, 8)

KeyboardInterrupt: 

In [ ]:
d